In [ ]:
# app_enhanced.py — hardened Dash app for CS 499 Databases enhancement
# - Uses env-based Mongo credentials (no hardcoded defaults)
# - Works even if DB is unavailable via CSV fallback (aac_shelter_outcomes.csv)
# - Defensive callbacks (handles empty/invalid data safely)
# - Clean layout, sortable/filterable table, pie chart, and map

import os
import base64
import pandas as pd

from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px

from animal_shelter import AnimalShelter

# -----------------------------------------------------------------------------
# Data access helpers
# -----------------------------------------------------------------------------
def _init_db() -> AnimalShelter | None:
    """
    Initialize AnimalShelter using env-based configuration.
    Accepts MONGO_URI or MONGO_HOST/PORT/USER/PASS with MONGO_DB/MONGO_COL.
    Returns None if connection fails (app will use CSV fallback).
    """
    try:
        mongo_uri = os.getenv("MONGO_URI")
        if mongo_uri:
            return AnimalShelter(mongo_uri=mongo_uri)
        # If URI not set, try discrete params. All must exist.
        host = os.getenv("MONGO_HOST")
        port = os.getenv("MONGO_PORT")
        user = os.getenv("MONGO_USER")
        pwd = os.getenv("MONGO_PASS")
        db = os.getenv("MONGO_DB") or "AAC"
        col = os.getenv("MONGO_COL") or "animals"
        if host and port and user and pwd:
            return AnimalShelter(
                host=host,
                port=int(port),
                username=user,
                password=pwd,
                db_name=db,
                collection_name=col,
            )
    except Exception as e:
        # Log to console; UI remains functional via CSV fallback
        print(f"[WARN] DB init failed: {e}")
    return None


def _read_records(db: AnimalShelter | None, query: dict | None = None, limit: int = 500) -> list[dict]:
    """
    Pull records from DB if available; otherwise return [] (caller can fallback to CSV).
    """
    if db is None:
        return []
    try:
        return db.read(query or {}, limit=limit)
    except Exception as e:
        print(f"[WARN] DB read failed: {e}")
        return []


def _load_csv_fallback(path: str = "aac_shelter_outcomes.csv") -> pd.DataFrame:
    """
    CSV fallback for local/offline use.
    """
    try:
        df_csv = pd.read_csv(path)
        # normalize columns used by map if they exist under different names
        # (no-op if already present)
        for alt_lat in ("latitude", "lat"):
            if "location_lat" not in df_csv.columns and alt_lat in df_csv.columns:
                df_csv["location_lat"] = df_csv[alt_lat]
                break
        for alt_lon in ("longitude", "lon", "long"):
            if "location_long" not in df_csv.columns and alt_lon in df_csv.columns:
                df_csv["location_long"] = df_csv[alt_lon]
                break
        return df_csv
    except Exception as e:
        print(f"[WARN] CSV fallback load failed: {e}")
        return pd.DataFrame()


def _to_dataframe(records: list[dict], csv_fallback: pd.DataFrame) -> pd.DataFrame:
    """
    Convert DB records to DataFrame; drop _id; if empty, use CSV fallback.
    """
    if records:
        df = pd.DataFrame.from_records(records)
        if "_id" in df.columns:
            df = df.drop(columns=["_id"], errors="ignore")
        return df
    return csv_fallback.copy()


# -----------------------------------------------------------------------------
# Bootstrap data
# -----------------------------------------------------------------------------
DB = _init_db()
CSV_FALLBACK = _load_csv_fallback()

INITIAL_DF = _to_dataframe(_read_records(DB, {}, limit=500), CSV_FALLBACK)
if INITIAL_DF.empty:
    # ensure table has some columns to initialize
    INITIAL_DF = pd.DataFrame(columns=["name", "breed", "outcome_type", "age_upon_outcome_in_weeks",
                                       "location_lat", "location_long"])

# Logo (optional)
def _load_logo(path: str = "grazioso_salvare_logo.png") -> str | None:
    try:
        with open(path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except Exception:
        return None

LOGO_B64 = _load_logo()

# Rescue filter definitions
RESCUE_MAP = {
    "Water Rescue": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
    "Mountain or Wilderness Rescue": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky"],
    "Disaster Rescue or Individual Tracking": ["German Shepherd", "Doberman Pinscher", "Bloodhound", "Rottweiler"],
}

# -----------------------------------------------------------------------------
# Dash app
# -----------------------------------------------------------------------------
app = Dash(__name__)

def layout():
    return html.Div([
        html.Center(html.B(html.H1("CS-340 / CS-499 Animal Outcomes Dashboard"))),
        (html.Img(
            src=f"data:image/png;base64,{LOGO_B64}",
            style={"height": "90px", "margin": "10px"}
        ) if LOGO_B64 else html.Div()),
        html.H6("Dashboard by Gonzalo Ramos"),
        html.Hr(),

        html.Div([
            dcc.RadioItems(
                id="filter-type",
                options=[
                    {"label": "Water Rescue", "value": "Water Rescue"},
                    {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                    {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster Rescue or Individual Tracking"},
                    {"label": "Reset", "value": "Reset"},
                ],
                value="Reset",
                labelStyle={"display": "inline-block", "marginRight": "15px"},
                inputStyle={"marginRight": "6px"}
            ),
            dcc.Store(id="data-store"),  # keep current table data for other callbacks
        ], style={"padding": "10px"}),

        dash_table.DataTable(
            id="datatable-id",
            columns=[{"name": c, "id": c} for c in INITIAL_DF.columns],
            data=INITIAL_DF.to_dict("records"),
            page_size=10,
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "left", "minWidth": "100px", "maxWidth": "300px", "whiteSpace": "normal"},
            style_header={"backgroundColor": "rgb(230,230,230)", "fontWeight": "bold"},
            row_selectable="single",
            sort_action="native",
            filter_action="native"
        ),

        html.Div(className="row", style={"display": "flex", "marginTop": "20px"}, children=[
            html.Div(id="graph-id", className="col s12 m6", style={"width": "50%"}),
            html.Div(id="map-id", className="col s12 m6", style={"width": "50%"}),
        ])
    ])

app.layout = layout

# -----------------------------------------------------------------------------
# Callbacks
# -----------------------------------------------------------------------------
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "columns"),
    Output("data-store", "data"),
    Input("filter-type", "value")
)
def update_table(filter_type: str):
    """
    Update table rows (and column schema) based on filter selection.
    Prefers DB; if no records, falls back to CSV.
    """
    try:
        if filter_type == "Reset":
            records = _read_records(DB, {}, limit=500)
        else:
            breeds = RESCUE_MAP.get(filter_type, [])
            query = {
                "breed": {"$in": breeds},
                "age_upon_outcome_in_weeks": {"$lte": 104}
            }
            records = _read_records(DB, query, limit=500)

        df = _to_dataframe(records, CSV_FALLBACK)
        cols = [{"name": c, "id": c} for c in df.columns]
        return df.to_dict("records"), cols, df.to_dict("records")
    except Exception as e:
        print(f"[WARN] update_table error: {e}")
        df = INITIAL_DF.copy()
        cols = [{"name": c, "id": c} for c in df.columns]
        return df.to_dict("records"), cols, df.to_dict("records")


@app.callback(
    Output("graph-id", "children"),
    Input("data-store", "data")
)
def update_graphs(view_data):
    """
    Pie chart of preferred breeds in current table view.
    """
    try:
        dff = pd.DataFrame(view_data) if view_data else INITIAL_DF
        if "breed" in dff.columns and not dff.empty:
            fig = px.pie(dff, names="breed", title="Preferred Breeds (current filter)")
            return [dcc.Graph(figure=fig)]
        return [html.P("No data available for chart.")]
    except Exception as e:
        print(f"[WARN] update_graphs error: {e}")
        return [html.P("Chart unavailable.")]


@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns")
)
def highlight_columns(selected_columns):
    if not selected_columns:
        return []
    return [{"if": {"column_id": c}, "background_color": "#D2F3FF"} for c in selected_columns]


@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
    Input("datatable-id", "derived_virtual_selected_rows"),
    State("data-store", "data")
)
def update_map(virtual_data, selected_rows, backing_store):
    """
    Show map marker for the selected row.
    Uses columns: location_lat, location_long, breed, name (if present).
    """
    try:
        # Prefer the DataTable's current view; fallback to store
        rows = virtual_data if virtual_data is not None else backing_store
        if not rows or not selected_rows:
            return html.Div()

        dff = pd.DataFrame(rows)
        row_idx = selected_rows[0]

        # Extract and validate coordinates
        lat = float(dff.iloc[row_idx].get("location_lat"))
        lon = float(dff.iloc[row_idx].get("location_long"))

        breed = str(dff.iloc[row_idx].get("breed", "Unknown"))
        name = str(dff.iloc[row_idx].get("name", "Unknown"))

        if pd.isna(lat) or pd.isna(lon):
            return html.P("Invalid geolocation data.")

        return [
            dl.Map(
                style={"width": "100%", "height": "500px"},
                center=[lat, lon],
                zoom=12,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[lat, lon],
                        children=[
                            dl.Tooltip(breed),
                            dl.Popup([html.H4("Animal Name"), html.P(name)])
                        ],
                    ),
                ],
            )
        ]
    except Exception as e:
        print(f"[WARN] update_map error: {e}")
        return html.P("Map unavailable.")

# -----------------------------------------------------------------------------
# Entrypoint
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Note: for GitHub Pages you’ll export static assets; for local dev:
    app.run_server(host="0.0.0.0", port=int(os.getenv("PORT", "8050")), debug=True)


Dash app running on http://127.0.0.1:19257/
